In [2]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [3]:
from fastembed import TextEmbedding
import numpy as np

In [7]:
query = 'I just discovered the course. Can I join now?'
model = 'jinaai/jina-embeddings-v2-small-en'

embedder = TextEmbedding(model_name=model, device='cpu')
# Generate the embedding for the query in numpy array format
embedding = np.array(list(embedder.embed(query))[0])
print(np.min(embedding))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

-0.11726373885183883


In [8]:
len(embedding)  # Length of the embedding vector

512

In [6]:
for model in TextEmbedding.list_supported_models():
    if model['model'] == 'jinaai/jina-embeddings-v2-small-en':
        print(model)
        break

{'model': 'jinaai/jina-embeddings-v2-small-en', 'sources': {'hf': 'xenova/jina-embeddings-v2-small-en', 'url': None, '_deprecated_tar_struct': False}, 'model_file': 'onnx/model.onnx', 'description': 'Text embeddings, Unimodal (text), English, 8192 input tokens truncation, Prefixes for queries/documents: not necessary, 2023 year.', 'license': 'apache-2.0', 'size_in_GB': 0.12, 'additional_files': [], 'dim': 512, 'tasks': {}}


In [9]:
doc = 'Can I still join the course after the start date?'

In [11]:
embedding.dot(list(embedder.embed(doc))[0])  # Compute the dot product with another document embedding

0.9008528895674548

In [12]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [13]:
doc_embeddings = list(embedder.embed([d['text'] for d in documents]))
len(doc_embeddings)

5

In [14]:
for i, vec in enumerate(doc_embeddings):
    print(f"Document {i} cosine similarity: {embedding.dot(vec)}")

Document 0 cosine similarity: 0.7629684696540238
Document 1 cosine similarity: 0.8182378150042889
Document 2 cosine similarity: 0.8085397398734189
Document 3 cosine similarity: 0.7133079015686243
Document 4 cosine similarity: 0.7304499234333598


In [15]:
for i, vec in enumerate(embedder.embed([d['question'] + ' ' + d['text'] for d in documents])):
    print(f"Document {i} cosine similarity: {embedding.dot(vec)}")

Document 0 cosine similarity: 0.8514543236908068
Document 1 cosine similarity: 0.8436594159113068
Document 2 cosine similarity: 0.8408287048502558
Document 3 cosine similarity: 0.7755157969663908
Document 4 cosine similarity: 0.8086007795043938


In [17]:
emb_lens = [{'name': model['model'], 'dim': model['dim']} for model in TextEmbedding.list_supported_models()]
print(min(emb_lens, key=lambda x: x['dim']))

{'name': 'BAAI/bge-small-en', 'dim': 384}


In [18]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
!docker run -d -p 6333:6333 -p 6334:6334 \
   -v "./qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


e716ae509ec82ed3c79c05fd30020c2328e021a733e6512c6d41375f56310e95


In [25]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [26]:
# Create the collection with specified vector parameters
client.create_collection(
    collection_name='machine-learning-zoomcamp',
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [27]:
points = []

for id, course in enumerate(documents_raw):
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['question'] + ' ' + doc['text'], model='BAAI/bge-small-en'),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

In [30]:
client.upsert(
    collection_name='machine-learning-zoomcamp',
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def search(query, limit=1):

    results = client.query_points(
        collection_name='machine-learning-zoomcamp',
        query=models.Document(
            text=query,
            model='BAAI/bge-small-en' 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [34]:
search(query)

QueryResponse(points=[ScoredPoint(id=1, version=0, score=0.806765, payload={'text': 'I’ve seen LinkedIn users list DataTalksClub as Experience with titles as:\nMachine Learning Fellow\nMachine Learning Student\nMachine Learning Participant\nMachine Learning Trainee\nPlease note it is best advised that you do not list the experience as an official “job” or “internship” experience since DataTalksClub did not hire you, nor financially compensate you.\nOther ways you can incorporate the experience in the following sections:\nOrganizations\nProjects\nSkills\nFeatured\nOriginal posts\nCertifications\nCourses\nBy Annaliese Bronz\nInteresting question, I put the link of my project into my CV as showcase and make posts to show my progress.\nBy Ani Mkrtumyan', 'section': 'Miscellaneous', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [33]:
query

'I just discovered the course. Can I join now?'